In [5]:
#!/usr/bin/env python
import logging
import os
from json import dumps
from textwrap import dedent
from typing import cast

import neo4j

from neo4j import GraphDatabase, basic_auth
from typing_extensions import LiteralString



url = os.getenv("NEO4J_URI", "bolt://124.220.2.168:7687")
username = os.getenv("NEO4J_USER", "neo4j")
password = os.getenv("NEO4J_PASSWORD", "copd1234")
neo4j_version = os.getenv("NEO4J_VERSION", "4")
database = os.getenv("NEO4J_DATABASE", "hod.db")

port = int(os.getenv("PORT", 8080))

driver = GraphDatabase.driver(url, auth=basic_auth(username, password))


In [6]:

def query(q: LiteralString) -> LiteralString:
    # this is a safe transform:
    # no way for cypher injection by trimming whitespace
    # hence, we can safely cast to LiteralString
    return cast(LiteralString, dedent(q).strip())

In [9]:
records, _, _ = driver.execute_query(
    query("""
        MATCH (n)-[r]->(m)
        RETURN n, r, m
        LIMIT $limit
    """),
    database_=database,
    routing_="r",
    limit=2
)
nodes = []
rels = []
for record in records:
    a = record
    break

In [16]:
a[0].element_id

'155731'

In [123]:

def serialize_node(node):
    return {
        "id": node.element_id,
        "name": node["name"],
        # "labels": list(node.labels)[0],
        # "content": node['content']
    }


In [121]:

def get_graph():
    records, _, _ = driver.execute_query(
        query("""
            MATCH (n)-[r]->(m)
            RETURN n, r, m
            LIMIT $limit
        """),
        database_=database,
        routing_="r",
        limit=5
    )
    nodes = []
    rels = []
    for record in records:
        a = record
        start_node = serialize_node(record[0])
        end_node = serialize_node(record[2])
        rel = record[1]
        # 添加起始节点和结束节点
        if start_node not in nodes:
            nodes.append(start_node)
        if end_node not in nodes:
            nodes.append(end_node)
        # 添加关系
        rels.append({"source": nodes.index(start_node), "target": nodes.index(end_node), "type": rel.type})
    return {"nodes": nodes, "links": rels}

In [122]:
get_graph()

{'nodes': [{'id': '155731', 'name': 'Alexander disease'},
  {'id': '155782', 'name': 'HP:0001645'},
  {'id': '155792', 'name': 'HP:0100716'},
  {'id': '155791', 'name': 'HP:0012332'},
  {'id': '155790', 'name': 'HP:0007481'},
  {'id': '155789', 'name': 'HP:0003307'}],
 'links': [{'source': 0, 'target': 1, 'type': 'clinical feature'},
  {'source': 0, 'target': 2, 'type': 'clinical feature'},
  {'source': 0, 'target': 3, 'type': 'clinical feature'},
  {'source': 0, 'target': 4, 'type': 'clinical feature'},
  {'source': 0, 'target': 5, 'type': 'clinical feature'}]}

In [142]:
#!/usr/bin/env python
import logging
import os
from contextlib import asynccontextmanager
from textwrap import dedent
from typing import Optional, cast

import neo4j
from fastapi import FastAPI, HTTPException
from fastapi.responses import FileResponse
from neo4j import AsyncGraphDatabase
from typing_extensions import LiteralString
driver = AsyncGraphDatabase.driver(url, auth=(username, password))
def get_search(q: Optional[str] = None):
    if q is None:
        return []
    records, _, _ = driver.execute_query(
        query(f"""
                MATCH (node)
                WHERE toLower(node.name) CONTAINS toLower('{q}')
                RETURN node
        """),
        database_=database,
        routing_="r",
    )
    return [serialize_node(record["node"]) for record in records]

In [143]:
get_search('syndrome')

C:\Users\xd\AppData\Local\Temp\ipykernel_27792\2725328850.py:17: RuntimeWarning: coroutine 'AsyncDriver.execute_query' was never awaited
  records, _, _ = driver.execute_query(


TypeError: cannot unpack non-iterable coroutine object

In [135]:
from flask import jsonify, current_app
def get_search():

    q = 'thalassemia'
    records, _, _ = driver.execute_query(
        query("""
            MATCH (node)
            WHERE toLower(node.name) CONTAINS toLower($name)
            RETURN node
        """),
        name=q,
        database_=database,
        routing_="r",
    )
    search_results = [serialize_node(record["node"]) for record in records]
        # 返回JSON响应
    with current_app.app_context():
        return jsonify(search_results)          
            


In [136]:
get_search()

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
the current application. To solve this, set up an application context
with app.app_context(). See the documentation for more information.

In [102]:
def get_search():
    try:
        q = 'thalassemia'
        records, _, _ = driver.execute_query(
            query("""
                MATCH (node)
                WHERE toLower(node.name) CONTAINS toLower($name)
                RETURN node
            """),
            name=q,
            database_=database,
            routing_="r",
        )
        return {"nodes": [serialize_node(record["node"]) for record in records]}
            # dumps([serialize_node(record["node"]) for record in records]),
           
    except KeyError:
        return []

In [124]:
def get_search():
    try:
        q = 'thalassemia'
        records, _, _ = driver.execute_query(
            query("""
                MATCH (node)
                WHERE toLower(node.name) CONTAINS toLower($name)
                RETURN node
            """),
            name=q,
            database_=database,
            routing_="r",
        )
        return dumps([serialize_node(record["node"]) for record in records])
    except KeyError:
        return []

In [125]:
get_search()

'[{"id": "156099", "name": "Beta-thalassemia"}, {"id": "156137", "name": "Alpha-thalassemia"}, {"id": "157937", "name": "Alpha-thalassemia-X-linked intellectual disability syndrome"}, {"id": "159853", "name": "Hereditary persistence of fetal hemoglobin-beta-thalassemia syndrome"}, {"id": "160854", "name": "Alpha-thalassemia-intellectual disability syndrome linked to chromosome 16"}, {"id": "161402", "name": "Dominant beta-thalassemia"}, {"id": "161414", "name": "Beta-thalassemia major"}, {"id": "161424", "name": "Beta-thalassemia-X-linked thrombocytopenia syndrome"}, {"id": "161425", "name": "Alpha-thalassemia-myelodysplastic syndrome"}, {"id": "161426", "name": "Hemoglobin C-beta-thalassemia syndrome"}, {"id": "161427", "name": "Hemoglobin E-beta-thalassemia syndrome"}, {"id": "161428", "name": "Delta-beta-thalassemia"}, {"id": "165689", "name": "Beta-thalassemia intermedia"}, {"id": "168486", "name": "Sickle cell-beta-thalassemia disease syndrome"}, {"id": "168705", "name": "Hemoglob

In [106]:
from flask import Flask, Response, request
def get_search():
    try:
        q = 'thalassemia'
        records, _, _ = driver.execute_query(
            query("""
                MATCH (node)
                WHERE toLower(node.name) CONTAINS toLower($name)
                RETURN node
            """),
            name=q,
            database_=database,
            routing_="r",
        )
        return Response(
            dumps([serialize_node(record["node"]) for record in records]),            
            mimetype="application/json"
        )


    except KeyError:
        return []